# Final Version 3

<b><i> Identified [Aim](#aim) of the project as, see link to later in document

### External Modules

In [1]:
import numpy as np
import pandas as pd
import ADRpy as ADR
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [2]:
plt.rcParams["figure.figsize"]=15,10 #Set plot size
plt.rcParams.update({'font.size': 16}) #set plot size font
plt.rcParams.update({"axes.grid" : True, "grid.color": "lightgrey"}) #Add grid lines to plot

### Conversions

In [3]:
def Knots(MS):
    return MS / 0.5144

def MS(Knots):
    return Knots * 0.5144

def radian(degree):
    return degree * np.pi/180

### Environment Parameters

In [4]:
g = 9.81        #Gravitational Constant
rho = 1.225

# Wind Speed / Direcion
#     Headwind +ve
#     Cross wind +ve from right
w_vel = 0
w_deg = 0

mu = 0.05    #Driction Coefficient to Runway Surface

#Possibly ovveride given rho (using ISA altitudes)
temp_c = 15     #Airfield OAT temperacture in celcius
alt_ft = 83     #Airfield (runway) altitude in feet
RunwayLength = 1650

In [5]:
# Vw = MS(5) #5 kt headwind
Vw = 0

<div class="alert alert-block alert-info">
Reasonable approximation of g to 9.81 as still very close to this. Density can be resonably approximated to 1.225 as very close to sea level given in relation. only change to the density will come from the temperature change, so use of ADRpy to calculate is ideal, on standard ISA non-offset (research into the offset and why used should be looked at)
</div>

### Aircraft Parameters

#### Airframe

In [6]:
m = 54000        #Mass

S = 122.6        #Wing Area
h_W = 3.38      #Height of wing above the ground

b = 34.1       #Wingspan

c = S/b       #Chord length

In [7]:
A = (b**2)/S
e = 0.85
a0 = 2*np.pi
alpha_0L = - radian(2)

def CL(alpha):
    alpha1 = radian(alpha)
    return a0 * ((np.pi*A*e)/(np.pi*A*e + a0)) * (alpha1 - alpha_0L)

#def Cl(alpha):
#    a0 = 2*np.pi
#    alpha0 = radian(2)
#    
#    return a0*(alpha + alpha0/57.3)

In [8]:
CD0 = 0.02

#### Engine

In [9]:
T = 120.6 * 1e3     #Thrust per engine
n_e = 2   #Number of engines (evenly distributed per wing (i.e. 2 engines = 1 per wing))

### Engine Failure Parameters  (likely not be necessary*)
<i>* Due to running a balanced field model, engine failure is automatically used in calculating this, as such the needed for a failure speed isn't required (this will be covered by the calculated V1 speed). `However, the engine that fails is a parameter that will likely be used and will be an input controlled by the overall Monte Carlo analysis`

In [10]:
V_fail = 0    #Speed at which failure occurs
E_Ident = 0   #Engine Number to fail

<hr style="height:1px;border-width:0;color:black;background-color:black">

### Key Functions

- Lift
- Drag
- Ground Effect
- Friction
- Induced Drag
- phi
<br>

* ODEs for:
    * Velocity
    * Distance travelled

<div class="alert alert-block alert-warning">
Check definition of lift when using total Cl (i.e. when tail is included as well, which S is needed)
</div>
<div class="alert alert-block alert-warning">
Define Drag components seperately (Profile, Induced, Friction etc.) so that ODEs for the different stages can be more easily defined
</div>

In [11]:
def Lift(V,alpha):
    return 0.5 * rho * V**2 * S * CL(alpha)

def Drag(V,alpha):
    return None

def D_i(V,alpha):
    return 0

def D(V,h,alpha):
    return 0.5 * rho * V**2 * S * (CD0 + phi(h) * (CL(alpha)**2)/(np.pi * e * A))

def D_f(V,alpha):
    R = m*g - Lift(V,alpha)
    return mu * R

def phi(h_alt):
    h = h_alt + h_W
    return ((16*h/b)**2)/((1+((16*h/b)**2)))

`Introduction to Flight pg 320` <br>
$\phi = \large\frac{(16 h/b)^{2}}{1+(16 h/b)^{2}}$

<hr style="height:1px;border-width:0;color:black;background-color:black">

### Calculating Stall Speed

For stall speed, the lift equals the weight of the aircraft. i.e $L=\frac{1}{2}\rho V^{2}C_{L}S=W=mg$
<br> 
Rearranging for V: $V= \sqrt{\frac{2mg}{\rho C_{L}S}}$

In [12]:
CL_max = CL(1.375)

V_s = np.sqrt((2*m*g)/(rho * S * CL_max))
Knots(V_s)

299.838881881468

### Calculating Rotation Speed

In [13]:
V2 = 1.2 * V_s
Knots(V2), V2

(359.80665825776157, 185.08454500779254)

#### Key Speeds:
- Vr : Speed at which rotation is initiated so that aircraft clears 35ft screen height
- V1 : Decision speed, maximum speed for Rejected Take Off
- V2 : Min climb speed OEO, must be reached by 35ft
- Vs : Stall Speed @ max CL
- VLOF : Aircraft leaves the ground

<div class="alert alert-block alert-info">
V1 is found by conducting the balanced field investigation for the parameters entered. <br><br> <a id='aim'></a><b> Aim to create a monte carlo analysis of balanced field lengths for a number of different aircraft parameters (should be able to input array for each of the paramters listed under Aircraft Parameters and Environmental Parameters sections above </b>
</div>

<hr style="height:1px;border-width:0;color:black;background-color:black">

In [ ]:
Failed_Engine = "N" #N - No failure, #L{0} - Left Failure & how many, #R{0} - Right Failure & how many

In [ ]:
#Total engine thrust (per side?)
#T_L = (n_e / 2) * T
#T_R = (n_e / 2) * T

Failed_Engine = Failed_Engine.upper()

if Failed_Engine == "N":
    T_L = T_R = (n_e / 2) * T
    
elif 'L' in Failed_Engine:
    e_p = n_e / 2
    T_L = (e_p - int(Failed_Engine[1:])) * T
    T_R = (n_e / 2) * T
else:
    e_p = n_e / 2
    T_L = (n_e / 2) * T
    T_R = (e_p - int(Failed_Engine[1:])) * T
T_L, T_R

# AEO (Euler Time-Stepping)

## Accelerate - Stop

### Accelerate

In [ ]:
%%time
V_abort = 150

def rollout(t,y):
    Vx = y[0]
    X = y[1]
    
    dVxdt = (1/m) * ((T_L + T_R) - D(Vx,0,0) - D_f(Vx,0))
    dXdt = Vx - Vw
    
    return np.array([dVxdt,dXdt])

def Abort(t,y):
    Vx = y[0]
    return Vx - MS(V_abort)
Abort.terminal = True

x0 = 0
v0 = 0
y0 = np.array([v0,x0])

sol_rollout = solve_ivp(rollout,[0,40],y0,rtol=1e-9,max_step=1e-3,events=[Abort])

In [ ]:
print('Vx : ', sol_rollout.y[0][-1])
print('X  : ', sol_rollout.y[1][-1])
print('t  : ', sol_rollout.t[-1])

In [ ]:
plt.plot(sol_rollout.t,Knots(sol_rollout.y[0]))
plt.plot([0,sol_rollout.t[-1]+1],[V_abort,V_abort])
plt.show()
plt.plot(sol_rollout.t,sol_rollout.y[1])
plt.show()

### Stopping

In [ ]:
%%time
def brake(t,y):
    Vx = y[0]
    X = y[1]
    
    dVxdt = -0.5 * g
    dXdt = Vx - Vw
    
    return np.array([dVxdt,dXdt])

def stop(t,y):
    Vx = y[0]
    return Vx
stop.terminal = True

x0 = sol_rollout.y[1][-1]
v0 = sol_rollout.y[0][-1]
y0 = np.array([v0,x0])

sol_brake = solve_ivp(brake,[0,40],y0,rtol=1e-9,max_step=1e-3,events=[stop])

In [ ]:
sol_brake.t[-1]

In [ ]:
plt.plot(sol_brake.t,Knots(sol_brake.y[0]))
plt.show()
plt.plot(sol_brake.t,sol_brake.y[1])
plt.show()

In [ ]:
print('Run stopped at {:.6f}'.format(sol_brake.t[-1]))
print('Run length     {:.6f}'.format(sol_brake.y[1][-1]))

## Take-off to 35ft

Notes:
 - For engine failure, thrust is dependent on the velocity, cut engine at speed just after V1

### Ground Run

In [ ]:
%%time

def rollout(t,y):
    Vx = y[0]
    X = y[1]
    
    dVxdt = (1/m) * ((T_L + T_R) - D(Vx,0,0) - D_f(Vx,0))
    dXdt = Vx - Vw
    
    return np.array([dVxdt,dXdt])

Vr = 170

def Vr_set(t,y):
    Vx = y[0]
    return Vx - MS(Vr)
Vr_set.terminal = True

x0 = 0
v0 = 0
y0 = np.array([v0,x0])

sol_rollout = solve_ivp(rollout,[0,50],y0,rtol=1e-9,max_step=1e-3,events=[Vr_set])

In [ ]:
print('Vx : ', sol_rollout.y[0][-1])
print('X  : ', sol_rollout.y[1][-1])

In [ ]:
plt.plot(sol_rollout.t,Knots(sol_rollout.y[0]))
plt.plot([0,sol_rollout.t[-1]+1],[Vr,Vr])
plt.show()
plt.plot(sol_rollout.t,sol_rollout.y[1])
plt.show()

### Transition / Rotate

Remember:
 - Friction still present in roatation
 - Change in AoA
 - Track Angle remains 0
 

In [ ]:
rotate_rate = 5

#V_LOF - Lift=Weight
def LOF(t,y):
    L = Lift(y[0],y[2])*np.cos(np.radians(y[2]))    
    W = m * g
    return L - W
LOF.terminal = True

def rotate(t,y):
    Vx = y[0]
    X = y[1]
    Alpha = y[2]
    
    dVxdt = (1/m) * ((T_L + T_R)*np.cos(np.radians(Alpha)) - D(Vx,0,Alpha) - D_f(Vx,Alpha))
    dAlphadt = rotate_rate
    dXdt = Vx - Vw
    
    return np.array([dVxdt,dXdt,dAlphadt])

In [ ]:
# initial conditions
x0  = sol_rollout.y[1][-1] # initial s
vx0 = sol_rollout.y[0][-1] # initial v
alpha0 = 0

# combine s and v into array y0
y0 = np.array([vx0,x0,alpha0])

sol_rotate = solve_ivp(rotate,[0,20],y0,rtol=1e-9,max_step = 1e-3,events=[LOF])

In [ ]:
plt.plot(sol_rotate.t,sol_rotate.y[0]) # vx
plt.show()
plt.plot(sol_rotate.t,sol_rotate.y[1]) # x
plt.show()
plt.plot(sol_rotate.t,sol_rotate.y[2]) # alpha
plt.show()

### Climb Out to 35ft

In [ ]:
alpha = theta = sol_rotate.y[2][-1]
alpha

In [ ]:
%%time

def ScreenHeight(t,y):
    return y[3] - 10.7
ScreenHeight.terminal = True

def f(t,y):
    Vx = y[0]  
    Vz = y[1]
    X = y[2]              # extract v from array y 
    H = y[3]
    
    V = np.sqrt(Vx**2 + Vz**2)

    dVxdt = (1/m) * ((T_L + T_R)*np.cos(np.radians(theta)) - D(V,0,theta)*np.cos(np.radians(theta)) - Lift(V,theta)*np.sin(np.radians(theta)))
    dVzdt = (1/m) * ((T_L + T_R)*np.sin(np.radians(theta)) + Lift(V,theta)*np.cos(np.radians(theta)) - m*g - D(V,0,theta)*np.sin(np.radians(theta)))
    dXdt = Vx - Vw
    dHdt = Vz
    
    return np.array([dVxdt, dVzdt, dXdt, dHdt]) # return array


# initial conditions
x0 = sol_rotate.y[1][-1] # initial s
vz0 = 0 # initial v
h0 = 0
vx0 = sol_rotate.y[0][-1]

# combine s and v into array y0
y0 = np.array([vx0,vz0,x0,h0])

sol_LOF = solve_ivp(f,[0,40],y0,rtol=1e-9,max_step = 1e-3,events=[ScreenHeight])

In [ ]:
plt.title('Vx')
plt.plot(sol_LOF.t,sol_LOF.y[0])
plt.show()
plt.title('Vz')
plt.plot(sol_LOF.t,sol_LOF.y[1])
plt.show()
plt.title('X')
plt.plot(sol_LOF.t,sol_LOF.y[2])
plt.show()
plt.title('H')
plt.plot(sol_LOF.t,sol_LOF.y[3])
plt.show()

In [ ]:
plt.plot(sol_LOF.t,sol_LOF.y[0],label = 'Vx')
plt.plot(sol_LOF.t,sol_LOF.y[1],label = 'Vz')
plt.legend()
plt.show()
plt.plot(sol_LOF.t,sol_LOF.y[2],label = 'X')
plt.plot(sol_LOF.t,sol_LOF.y[3],label = 'H')
plt.legend()
plt.show()

In [ ]:
sol_LOF.y[2][-1]

In [ ]:
Knots(sol_LOF.y[0][-1])

In [ ]:
Knots(sol_LOF.y[1][-1])

In [ ]:
np.sqrt((Knots(sol_LOF.y[0][-1]))**2+(Knots(sol_LOF.y[1][-1]))**2)

In [ ]:
at = np.arctan(sol_LOF.y[1][-1]/sol_LOF.y[0][-1])

In [ ]:
np.degrees(at)

pitch angle / track angle / speed / speed at screen height

## V1 AEO Acc-Stop

In [ ]:
%%time
V_abort = 150

def accel(t,y):
    Vx = y[0]
    X = y[1]
    
    dVxdt = (1/m) * ((T_L + T_R) - D(Vx,0,0) - D_f(Vx,0))
    dXdt = Vx - Vw
    return np.array([dVxdt,dXdt])

def brake(t,y):
    Vx = y[0]
    X = y[1]
    
    dVxdt = -0.5 * g
    dXdt = Vx - Vw
    
    return np.array([dVxdt,dXdt])

def Abort(t,y):
    Vx = y[0]
    return Vx - MS(V_abort)
Abort.terminal = True

def stop(t,y):
    Vx = y[0]
    return Vx
stop.terminal = True



x0 = 0
v0 = 0
y0 = np.array([v0,x0])
sol_rollout = solve_ivp(rollout,[0,40],y0,rtol=1e-9,max_step=1e-3,events=[Abort])

x0 = sol_rollout.y[1][-1]
v0 = sol_rollout.y[0][-1]
y0 = np.array([v0,x0])
sol_brake = solve_ivp(brake,[0,40],y0,rtol=1e-9,max_step=1e-3,events=[stop])

In [ ]:
sol_brake.y[1][-1]

In [ ]:
def find_V1(V_t):
    def V1Abort(t,y):
        Vx = y[0]
        return Vx - MS(V_t)
    V1Abort.terminal = True

    print(V_t)
    
    x0 = 0
    v0 = 0
    y0 = np.array([v0,x0])
    sol_accel = solve_ivp(accel,[0,100],y0,rtol=1e-9,max_step=1e-3,events=[V1Abort])

    x0 = sol_accel.y[1][-1]
    v0 = sol_accel.y[0][-1]
    y0 = np.array([v0,x0])
    sol_brake = solve_ivp(brake,[0,100],y0,rtol=1e-9,max_step=1e-3,events=[stop])
    
    return sol_brake.y[1][-1] - RunwayLength

In [ ]:
find_V1(150)

In [ ]:
from scipy.optimize import newton
newton(find_V1,160, tol=1e-0)

In [ ]:
V_abort = newton(find_V1,160,tol=1e-0)

x0 = 0
v0 = 0
y0 = np.array([v0,x0])
sol_rollout = solve_ivp(rollout,[0,100],y0,rtol=1e-9,max_step=1e-3,events=[Abort])

x0 = sol_rollout.y[1][-1]
v0 = sol_rollout.y[0][-1]
y0 = np.array([v0,x0])
sol_brake = solve_ivp(brake,[0,40],y0,rtol=1e-9,max_step=1e-3,events=[stop])

In [ ]:
sol_brake.y[1][-1]

In [ ]:
V_abort = np.floor(V_abort)

x0 = 0
v0 = 0
y0 = np.array([v0,x0])
sol_rollout = solve_ivp(rollout,[0,100],y0,rtol=1e-9,max_step=1e-3,events=[Abort])

x0 = sol_rollout.y[1][-1]
v0 = sol_rollout.y[0][-1]
y0 = np.array([v0,x0])
sol_brake = solve_ivp(brake,[0,40],y0,rtol=1e-9,max_step=1e-3,events=[stop])
sol_brake.y[1][-1]

In [ ]:
sol_rollout.t[-1]

# V1 Acc-Go
## Calc Vr

To calculate Vr, V2 is set at 1.2*Vs (the stall speed), this is to be reached at 35ft above the ground. Vr is the rotatation speed needed in order to obatain this requirement.
So to calculate Vr we look at the climb out to 35ft

Vr is rotation speed such that aircraft leaves ground at climb angle at V_LOF

In [ ]:
theta = 12

In [ ]:
%%time

def ScreenHeight(t,y):
    return y[3] - 10.7
ScreenHeight.terminal = True

def f(t,y):
    Vx = y[0]  
    Vz = y[1]
    X = y[2]              # extract v from array y 
    H = y[3]
    
    V = np.sqrt(Vx**2 + Vz**2)

    dVxdt = (1/m) * ((T_L + T_R)*np.cos(np.radians(theta)) - D(V,0,theta)*np.cos(np.radians(theta)) - Lift(V,theta)*np.sin(np.radians(theta)))
    dVzdt = (1/m) * ((T_L + T_R)*np.sin(np.radians(theta)) + Lift(V,theta)*np.cos(np.radians(theta)) - m*g - D(V,0,theta)*np.sin(np.radians(theta)))
    dXdt = Vx
    dHdt = Vz
    
    return np.array([dVxdt, dVzdt, dXdt, dHdt]) # return array

In [ ]:
def V2_search(V_t):
    # initial conditions
    x0 = sol_rotate.y[1][-1] # initial s
    vz0 = 0 # initial v
    h0 = 0
    vx0 = V_t

    # combine s and v into array y0
    y0 = np.array([vx0,vz0,x0,h0])
    print(y0)
    sol_t = solve_ivp(f,[0,10],y0,rtol=1e-9,max_step = 1e-3,events=[ScreenHeight])
    return np.sqrt(sol_t.y[0][-1]**2 + sol_t.y[1][-1]**2) - V2

In [ ]:
V2

In [ ]:
from scipy.optimize import newton

In [ ]:
V_lof = newton(V2_search,V2)
V_lof

In [ ]:
V2_search(V_lof)

V1, Vr, VLOF, V2

In [ ]:
Lift(V_lof,theta) - m*g

In [ ]:
theta

In [ ]:
sol_rotate.y[0]

V2 gives Vlof, Vlof gives Vr?

# _

climb angle needed so that aircraft vlof is when aircrfat leaves ground i.e. L component = W, and V2min is met at 35ft

Vr is rotation initiated such that climb angle met at LOF

V1 min speed needed such that takeoff can be rreached OEO

V2 is a given at 1.2 * Vs

In [ ]:
def ScreenHeight(t,y):
    return y[3] - 10.7
ScreenHeight.terminal = True

def f_OEI(t,y):
    Vx = y[0]  
    Vz = y[1]
    X = y[2]              # extract v from array y 
    H = y[3]
    
    V = np.sqrt(Vx**2 + Vz**2)
    
    dVxdt = (1/m) * ((0 + T_R)*np.cos(np.radians(theta)) - D(V,0,theta)*np.cos(np.radians(theta)) - Lift(V,theta)*np.sin(np.radians(theta)))
    dVzdt = (1/m) * ((0 + T_R)*np.sin(np.radians(theta)) + Lift(V,theta)*np.cos(np.radians(theta)) - m*g - D(V,0,theta)*np.sin(np.radians(theta)))
    dXdt = Vx
    dHdt = Vz
    
    return np.array([dVxdt, dVzdt, dXdt, dHdt]) # return array

In [ ]:
V2

In [ ]:
V_lof = np.sqrt((2*m*g)/(rho * S * CL(10)))
Knots(V_lof), V_lof, Lift(V_lof,10)*np.cos(radian(10))

In [ ]:
def climb(a):
    # initial conditions
    global theta
    theta = a
    x0 = sol_rotate.y[1][-1] # initial s
    vz0 = 0 # initial v
    h0 = 0
    vx0 = np.sqrt((2*m*g)/(rho * S * CL(a)))

    # combine s and v into array y0
    y0 = np.array([vx0,vz0,x0,h0])
    print(y0)
    sol_t = solve_ivp(f_OEI,[0,10],y0,rtol=1e-9,max_step = 1e-3,events=[ScreenHeight])
    return np.sqrt(sol_t.y[0][-1]**2 + sol_t.y[1][-1]**2) - V2

In [ ]:
climb(10)

In [ ]:
climb(8.9)

In [ ]:
climb(10.5)

In [ ]:
from scipy.optimize import newton

In [ ]:
newton(climb,10)

In [ ]:
np.sqrt((2*m*g)/(rho * S * CL(9.9)))

In [ ]:
climb_angle = np.round(newton(climb,10),1)
print(climb_angle)
# initial conditions
x0 = sol_rotate.y[1][-1] # initial s
vz0 = 0 # initial v
h0 = 0
vx0 = np.sqrt((2*m*g)/(rho * S * CL(climb_angle)))

# combine s and v into array y0
y0 = np.array([vx0,vz0,x0,h0])
sol_c = solve_ivp(f,[0,10],y0,rtol=1e-9,max_step = 1e-3,events=[ScreenHeight])

In [ ]:
theta = climb_angle
V_lof = np.sqrt((2*m*g)/(rho * S * CL(climb_angle)))

In [ ]:
V_lof

In [ ]:
Knots(V_lof)

In [ ]:
plt.title('Vx')
plt.plot(sol_c.t,sol_c.y[0])
plt.show()
plt.title('Vz')
plt.plot(sol_c.t,sol_c.y[1])
plt.show()
plt.title('V')
plt.plot(sol_c.t,np.sqrt(sol_c.y[0]**2+sol_c.y[1]**2))
plt.show()
plt.title('X')
plt.plot(sol_c.t,sol_c.y[2])
plt.show()
plt.title('H')
plt.plot(sol_c.t,sol_c.y[3])
plt.show()

## Vr

In [ ]:
rotate_rate = 3

def V_LOF(t,y):
    return y[0] - V_lof
V_LOF.terminal = True

def rotate_OEI(t,y):
    Vx = y[0]
    X = y[1]
    Alpha = y[2]
    
    dVxdt = (1/m) * ((0 + T_R)*np.cos(np.radians(Alpha)) - D(Vx,0,Alpha) - D_f(Vx,Alpha))
    dXdt = Vx - Vw
    dAlphadt = rotate_rate
    
    return np.array([dVxdt,dXdt,dAlphadt])

In [ ]:
def rotation(Vr):
    # initial conditions
    x0  = sol_rollout.y[1][-1] # initial s
    vx0 = MS(Vr)
    alpha0 = 0

    # combine s and v into array y0
    y0 = np.array([vx0,x0,alpha0])
    print(y0)
    sol_rot = solve_ivp(rotate_OEI,[0,40],y0,rtol=1e-9,max_step = 1e-3,events=[V_LOF])
    return sol_rot.y[2][-1] - climb_angle


In [ ]:
Vr = newton(rotation,150)

In [ ]:
Vr, MS(Vr)

In [ ]:
Knots(V_lof)

In [ ]:
# initial conditions
x0  = sol_rollout.y[1][-1] # initial s
vx0 = MS(Vr)
alpha0 = 0

# combine s and v into array y0
y0 = np.array([vx0,x0,alpha0])
sol_rotate = solve_ivp(rotate_OEI,[0,40],y0,rtol=1e-9,max_step = 1e-3,events=[V_LOF])
sol_rotate.y[0][-1]

In [ ]:
plt.plot(sol_rotate.t,sol_rotate.y[0]) # vx
plt.show()
plt.plot(sol_rotate.t,sol_rotate.y[1]) # x
plt.show()
plt.plot(sol_rotate.t,sol_rotate.y[2]) # alpha
plt.show()

## V1

min speed needed to safely continue takeoff, i.e what speed needed to continue with only one engine left
i have Vr, V2, angle to climb out to obstacle with OEI

speed needed to be able to get into the air before end of runway

V1 <= Vr

If AEO take-off run length is greater than runway available, runway airfield not suitable, different airport or weight required.
same whill apply if V1 > Vr

What speed is needed so that possible to accelerate upto Vr, rotate and climb before end of runway
min speed therefore max limit (total length equals total available

 engine fails at V1, pick a speed for V1, attempt the takeoff if run too long increase V1, repeat until within length of runway.

In [ ]:
V1 = Vr
print(V1)
print(Vr)
print(V_lof)
#Speeds
#   Vr
#   Vlof

#Stages
#  Accel to V1 (AEO)
def toV1(t,y):
    Vx = y[0]
    X = y[1]
    
    dVxdt = (1/m) * ((T_L + T_R) - D(Vx,0,0) - D_f(Vx,0))
    dXdt = Vx - Vw
    
    return np.array([dVxdt,dXdt])

#  Accel to Vr (OEI)
def V1Vr(t,y):
    Vx = y[0]
    X = y[1]
    
    dVxdt = (1/m) * ((0 + T_R) - D(Vx,0,0) - D_f(Vx,0))
    dXdt = Vx - Vw
    
    return np.array([dVxdt,dXdt])

#  Rotation (OEI)
def rotate(t,y):
    Vx = y[0]
    X = y[1]
    Alpha = y[2]
    
    dVxdt = (1/m) * ((T_L + T_R)*np.cos(np.radians(Alpha)) - D(Vx,0,Alpha) - D_f(Vx,Alpha))
    dXdt = Vx - Vw
    dAlphadt = rotate_rate
    
    return np.array([dVxdt,dXdt,dAlphadt])

#  Climb (OEI)
def climbout(t,y):
    Vx = y[0]  
    Vz = y[1]
    X = y[2]               
    H = y[3]
    
    V = np.sqrt(Vx**2 + Vz**2)
    dVxdt = (1/m) * ((T_L + T_R)*np.cos(np.radians(theta)) - D(V,0,theta)*np.cos(np.radians(theta)) - Lift(V,theta)*np.sin(np.radians(theta)))
    dVzdt = (1/m) * ((T_L + T_R)*np.sin(np.radians(theta)) + Lift(V,theta)*np.cos(np.radians(theta)) - m*g - D(V,0,theta)*np.sin(np.radians(theta)))
    dXdt = Vx - Vw
    dHdt = Vz
    
    return np.array([dVxdt, dVzdt, dXdt, dHdt]) 

#Constraints
#  V1
def set_V1(t,y):
    Vx = y[0]
    return Vx - MS(V1)
set_V1.terminal = True

#  Vr
def set_Vr(t,y):
    Vx = y[0]
    return Vx - MS(Vr)
set_Vr.terminal = True

#  Vlof
def V_LOF(t,y):
    return y[0] - V_lof
V_LOF.terminal = True

#  Screen Height (35ft)
def ScreenHeight(t,y):
    return y[3] - 10.7
ScreenHeight.terminal = True

#Calculations
#  upto V1
print('V1')
x0 = 0
v0 = 0
y0 = np.array([v0,x0])

sol_V1 = solve_ivp(toV1,[0,60],y0,rtol=1e-9,max_step=1e-3,events=[set_V1])

#  V1 to Vr
if Vr != V1:
    print('V1Vr')
    x0 = sol_V1.y[1][-1]
    v0 = sol_V1.y[0][-1]
    y0 = np.array([v0,x0])

    sol_Vr = solve_ivp(V1Vr,[0,60],y0,rtol=1e-9,max_step=1e-3,events=[set_Vr])
else:
    print('Vr = V1')
    sol_Vr = sol_V1
        
#  Rotation (Vr to Vlof)
print('rotate')
x0  = sol_Vr.y[1][-1]
vx0 = sol_Vr.y[0][-1]
alpha0 = 0
y0 = np.array([vx0,x0,alpha0])

sol_VrVlof = solve_ivp(rotate,[0,60],y0,rtol=1e-9,max_step = 1e-3,events=[V_LOF])

#  Climb (Vlof to V2)
print('climb')
# alpha = theta = sol_rotate.y[2][-1]

x0 = sol_VrVlof.y[1][-1]
vx0 = sol_VrVlof.y[0][-1]
vz0 = 0
h0 = 0
y0 = np.array([vx0,vz0,x0,h0])

sol_climb = solve_ivp(climbout,[0,60],y0,rtol=1e-9,max_step = 1e-3,events=[ScreenHeight])

In [ ]:
sol_V1.y[0][-1], sol_V1.y[1][-1]

In [ ]:
sol_Vr.y[0][-1], sol_Vr.y[1][-1]

In [ ]:
sol_VrVlof.y[0][-1], sol_VrVlof.y[1][-1], sol_VrVlof.y[2][-1]

In [ ]:
sol_climb.y[0][-1], sol_climb.y[1][-1], sol_climb.y[2][-1], sol_climb.y[3][-1]

In [ ]:
plt.title('Vx')
plt.plot(sol_climb.t,sol_climb.y[0])
plt.show()
plt.title('Vz')
plt.plot(sol_climb.t,sol_climb.y[1])
plt.show()
plt.title('X')
plt.plot(sol_climb.t,sol_climb.y[2])
plt.show()
plt.title('H')
plt.plot(sol_climb.t,sol_climb.y[3])
plt.show()

V2 2.4% climb one engine

percentage climb rate to angle

need sufficient excess thrust to meet 

In [ ]:
V2

V2 OEI min climb gradient = 2.4%, what is the speed?

In [ ]:
min_a = np.round(np.arctan(24/1000) * 180/np.pi,3)
min_a

In [ ]:
Lift(350,min_a)*np.cos(min_a) - m*g

In [ ]:
Knots(V_lof)

In [ ]:
V_s

In [ ]:
V2

In [ ]:
V_lof

l = (0.5 * rho * V**2 * Cl(a)) *np.cos(a) = m*g

In [ ]:
V = np.sqrt(((m*g)/np.cos(a))/(0.5*rho*Cl(a)))